<a href="https://colab.research.google.com/github/anushkagupta1307/IR2022_A1_27/blob/main/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount the data files from google drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import necessary libraries
import pydrive
import re
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk


In [ ]:
#nltk library for preprocessing
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


**Question 1(A) and 1(B)**

In [ ]:
#This method takes the Data files as input, does preprocessing steps on the data and then creates a unigram inverted index
import os
def preprocessing_indexing():
  word_dict=dict()
  doc_docID_mapping=dict()

  directory = '/content/drive/MyDrive/Humor,Hist,Media,Food'

  count=0
  docID=1
  for filename in os.scandir(directory):
      flag=0
      if filename.is_file():
          #print(filename.path)
          ccfile = open(filename.path, "r",encoding="utf8", errors="surrogateescape")

          doc_docID_mapping[docID]=filename.path

        
          for aline in ccfile:
              #Preprocessing steps included- 
              # Convert to lowercase
              aline = aline.lower()
              # Remove numerics
              aline = re.sub(r'\d+','',aline)
              # Remove whitespaces
              aline = re.sub(r'[^\w\s]','', aline)
              # Remove special characters
              aline= re.sub('[@_!#$%^&*()<>?/\|}{~:]','',aline)
              aline = aline.strip()
              #Remove stopwords
              aline = remove_stopwords(aline)

              #Convert to tokens
              words = word_tokenize(aline)
              
              #Create a dictionary for the index - the dictionary has the Root word(Stemming) as key and list of docIds as the value
              for w in words:
                  #If root word not already present in the dictionary then create a new key
                  if ps.stem(w) not in word_dict:
                    list1=[docID]
                    word_dict[ps.stem(w)]=list1
                    break;
                  else:
                    #else append the docId to the already existing list of docIds for the given word
                    if docID not in word_dict.get(ps.stem(w)):
                      word_dict[ps.stem(w)].append(docID)
                    
                    break;

                  print(ps.stem(w))
          
          ccfile.close()
          
          docID=docID+1
  totalDOCS=docID-1        
  return word_dict,doc_docID_mapping,totalDOCS

In [ ]:
word_dict=dict()
doc_docID_mapping=dict()
word_dict,doc_docID_mapping,totalDOCS=preprocessing_indexing()

**Question 1(C)**

In [ ]:
#This method is used for preprocessing data
def preprocess(word):
      #Convert to lowercase
      word = word.lower()
      #Remove numerics
      word = re.sub(r'\d+','',word)
      #Remove whitespaces
      word = re.sub(r'[^\w\s]','', word)
      #Remove special characters
      word= re.sub('[@_!#$%^&*()<>?/\|}{~:]','',word)
      word = word.strip()
      #remove stopwords
      word = remove_stopwords(word)
      
      #Stemming
      return ps.stem(word)

**x OR y**

In [ ]:
#This method takes a List of DocIds to which word x belongs and a list of DocIds to which word y belongs and then returns the resultant list after OR
# operation and the number of comparisons done to form the resultant list
def xory(listx,listy):
  
  #get the size of the two lists
  size_1 = len(listx)
  size_2 = len(listy)
    
  #Create a result list
  res = []
  i, j = 0, 0
  number_of_comparison=0
  
  #Iterate and compare
  while i < size_1 and j < size_2:
      #If element of list1 is lesser than list2 then add the element to result and increment pointer for list1- also increment the number of comparisons
      if listx[i] < listy[j]:
        if listx[i] not in res:
          res.append(listx[i])
        i += 1
        number_of_comparison=number_of_comparison+1
    
      #else append the element of list2 to the result and increment the list2 pointer- also increment the number of comparisons
      else:
        if listy[j] not in res:
          res.append(listy[j])
        j += 1
        number_of_comparison=number_of_comparison+1
    
  res = res + listx[i:] + listy[j:]

  #Return the result list and the number of comparisons done
  return res,number_of_comparison





**xANDy**

In [ ]:
#This method takes a List of DocIds to which word x belongs and a list of DocIds to which word y belongs and then returns the resultant list after AND
# operation and the number of comparisons done to form the resultant list
def xandy(listx,listy):
    
    #Create a result list
    res = []
    i, j = 0, 0

    #get the size of the two lists
    m = len(listx)
    n = len(listy)
    number_of_comparison=0
    while i < m and j < n:
        #If the element of list1 is smaller than list2 then increment the pointer to list1 and increase the no. of comparisons
        if listx[i] < listy[j]:
            i += 1
            number_of_comparison=number_of_comparison+1
        #else increment the pointer to list2 and increment the no. of comparisons
        elif listy[j] < listx[i]:
            j+= 1
            number_of_comparison=number_of_comparison+1
        #only if the element exists in both the lists then add it to the result list and increment both pointers
        else:
            res.append(listy[j])
            j += 1
            i += 1
    #Return the result list and the number of comparisons done
    return res,number_of_comparison

**xAnd Not y**

In [ ]:
#This method takes a List of DocIds to which word x belongs and a list of DocIds to which word y belongs and then returns the resultant list after AND NOT
# operation and the number of comparisons done to form the resultant list
def xandnoty(listx,listy,totalDOCS):

    #Get the list in which y does Not belong- subtract the lists to which y belongs from total docs
    newlisty=[]
    for i in range(1,totalDOCS+1,1):
      if i not in listy:
        newlisty.append(i)
    
    #create a result list
    res = []
    number_of_comparison=0
    i, j = 0, 0

    #Get the size of both the lists
    m = len(listx)
    n = len(newlisty)
    while i < m and j < n:
        #If the element of list1 is smaller than list2 then increment the pointer to list1 and increase the no. of comparisons 
        if listx[i] < newlisty[j]:
            i += 1
            number_of_comparison=number_of_comparison+1
        #else increment the pointer to list2 and increment the no. of comparisons
        elif newlisty[j] < listx[i]:
            j+= 1
            number_of_comparison=number_of_comparison+1
        #only if the element exists in x and in NewList y (Not y) then add it to the result list and increment both pointers
        else:
            res.append(newlisty[j])
            j += 1
            i += 1
            number_of_comparison=number_of_comparison+1
    #Return the result list and the number of comparisons done
    return res,number_of_comparison

**x OR NOT y**

In [ ]:
#This method takes a List of DocIds to which word x belongs and a list of DocIds to which word y belongs and then returns the resultant list after OR NOT
# operation and the number of comparisons done to form the resultant list
def xornoty(listx,listy,totalDOCS):
 
 #Get the list in which y does Not belong- subtract the lists to which y belongs from total docs
  newlisty=[]
  for i in range(1,totalDOCS+1,1):
      if i not in listy:
        newlisty.append(i)

  #Get the size of both the lists
  number_of_comparison=0
  size_1 = len(listx)
  size_2 = len(newlisty)
   #create a result list
  res = []
  i, j = 0, 0
    
  while i < size_1 and j < size_2:
      #If element of list1 is lesser than newlist y then add the element to result and increment pointer for list1- also increment the number of comparisons
      if listx[i] < newlisty[j]:
        if listx[i] not in res:
          res.append(listx[i])
        i += 1
        number_of_comparison=number_of_comparison+1
      #else append the element of newlisty to the result and increment the list2 pointer- also increment the number of comparisons
      else:
        if newlisty[j] not in res:
          res.append(newlisty[j])
        j += 1
        number_of_comparison=number_of_comparison+1
    
  res = res + listx[i:] + newlisty[j:]
  #Return the result list and the number of comparisons done
  return res,number_of_comparison





**Question 1(C)**

In [ ]:
#This block takes two words as input and an operator and performs the operation on the words and gives the result list of doc ids
x=input("Please enter first word ::  ")
y=input("Please enter second word ::  ")

#Preprocess the words
x= preprocess(x)
y= preprocess(y)
#Get the list of docIds to which the words belong from the index
listx= word_dict.get(x)
listy=word_dict.get(y)

#take the operator input 
op=input("Please enter the operation :: ")

#If the operation is OR call the xORy method
if op=="OR" or op=="or":
  res,number_of_comparison=xory(listx,listy)

#If the operation is AND call the xORy method
elif op=="AND" or op=="and":
  res,number_of_comparison=xandy(listx,listy)

#If the operation is OR NOT call the xORy method
elif op=="ORNOT" or op=="ornot" or op=="OR NOT" or op=="or not":
  res,number_of_comparison=xornoty(listx,listy,totalDOCS)

#If the operation is AND NOT call the xORy method
elif op=="ANDNOT" or op=="andnot" or op=="AND NOT" or op=="and not":
  res,number_of_comparison=xandnoty(listx,listy,totalDOCS)

#Print the number of documents and the number of comparisons and the document names in the result

print("Number of documents matched: ",len(res))
print("No. of comparisons required: ", number_of_comparison)
if(len(res)!=0):
  for i in range(len(res)):
    print("doc name ::  ",doc_docID_mapping[res[i]])
else:
  print("No matching Document!!!")

#Sprockets
#Washington

Please enter first word ::  Sprockets
Please enter second word ::  Washington
Please enter the operation :: or
Number of documents matched:  23
No. of comparisons required:  4
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/newconst.hum
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/onetoone.hum
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/ohandre.hum
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/dieter.txt
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/poll2res.hum
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/reagan.hum
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/socecon.hum
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/spydust.hum
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/lawyer.jok
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/misc.1
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/childhoo.jok
doc name ::   /content/drive/MyDrive/Humor,Hist

**Question 1(D)**

In [ ]:
#Preprocessing the data
def preprocess(word):
      #Convert to lowercase
      word = word.lower()
      #Remove numerics
      word = re.sub(r'\d+','',word)
      #Remove whitespaces
      word = re.sub(r'[^\w\s]','', word)
      #Remove special characters
      word= re.sub('[@_!#$%^&*()<>?/\|}{~:]','',word)
      word = word.strip()
      #Remove stopwords
      word = remove_stopwords(word)
      
      #Return the root word
      return ps.stem(word)

In [ ]:
import sys
import re
#take number of queries from user
total_number=int(input("Please enter the number of queries ::  "))

for i in range(total_number):
  query=input("Please enter the query ::  ")
  query=query.split()

#Preprocessing on input query
  list_words=[]
  for word in query:
        word = word.lower()
        word = re.sub(r'\d+','',word)
        word = re.sub(r'[^\w\s]','', word)
        word= re.sub('[@_!#$%^&*()<>?/\|}{~:]','',word)
        word = word.strip()
        word = remove_stopwords(word)

        if word!='':
          list_words.append(word)
  
  print(list_words)

  
  #Take the lis tof operations from the user
  list_of_operations=[]
  for i in range(1,len(list_words),1):
      operation=input("Please enter one operation.(OR/OR NOT/AND/AND NOT) ::  ")
      list_of_operations.append(operation)
  
  #Get the docids for word1 from the index

  if len(list_words) ==0:
    print("No word in the phrase query or all stopwords in the query")
    sys.exit("No Valid Word found in the phras query")
 
  word1=list_words[0]
  list_word1=word_dict.get(word1)
  
  #If the word doesnt exists in the index handle the error scenario
  if list_word1 is None:
    print("Cannot find the word ", word1,"in the dictionary")
    sys.exit("Cannot find word")

  if len(list_words) ==1:
    print("Only Single word in the query, hence no further operation needed")
    print("Number of Documents retrieved is ::  ",len(list_word1))
    for i in range(len(list_word1)):
        print("doc name ::  ",doc_docID_mapping[list_word1[i]])
    sys.exit("Number of Document retrieved for the same is given above")

    
  #Get the docids for word2 from the index
  word2=list_words[1]
  word2=preprocess(word2)
  list_word2=word_dict.get(word2)

  #If the word doesnt exists in the index handle the error scenario
  
  if list_word2 is None:
    print("Cannot find the word ", word2,"in the dictionary")
    sys.exit("Cannot find word")
    
  
  total_comparisons=0
  
  #Call the necessary method based on the operation and add the result to list of docIds and the number of comparisons
  #Done for first two words

  if(list_of_operations[0]=='OR' or list_of_operations[0]=='or'):
    res,c1=xory(list_word1,list_word2)
    total_comparisons=total_comparisons+c1

  elif(list_of_operations[0]=='OR NOT' or list_of_operations[0]=='or not' or list_of_operations[0]=='ORNOT' or list_of_operations[0]=='ornot'):
    res,c2=xornoty(list_word1,list_word2,totalDOCS)
    total_comparisons=total_comparisons+c2

  elif(list_of_operations[0]=='AND NOT' or list_of_operations[0]=='and not' or list_of_operations[0]=='ANDNOT' or list_of_operations[0]=='andnot'):
    res,c3=xandnoty(list_word1,list_word2,totalDOCS)
    total_comparisons=total_comparisons+c3

  elif(list_of_operations[0]=='AND' or list_of_operations[0]=='and'):
    res,c4=xandy(list_word1,list_word2)
    total_comparisons=total_comparisons+c4

  #Now iterate on the rest of the words
  #The result of first two is now combined with the third word and so on..
  for i in range(1,len(list_of_operations),1):

    nextword=list_words[i+1]
    #Preprocess the next word
    nextword=preprocess(nextword)
    #get the docids for next word from the index
    list_wordnext=word_dict.get(nextword)
    if(list_wordnext is None):
      print("Cannot find the word ", word2,"in the dictionary")
      sys.exit("Cannot find word")

    #perform the operation and add the list of docids and the number of comparisons accordingly

    if(list_of_operations[0]=='OR' or list_of_operations[0]=='or'):
      res,c5=xory(res,list_wordnext)
      total_comparisons=total_comparisons+c5

    elif(list_of_operations[0]=='OR NOT' or list_of_operations[0]=='or not' or list_of_operations[0]=='ORNOT' or list_of_operations[0]=='ornot'):
      res,c6=xornoty(res,list_wordnext,totalDOCS)
      total_comparisons=total_comparisons+c6

    elif(list_of_operations[0]=='AND NOT' or list_of_operations[0]=='and not' or list_of_operations[0]=='ANDNOT' or list_of_operations[0]=='andnot'):
      res,c7=xandnoty(res,list_wordnext,totalDOCS)
      total_comparisons=total_comparisons+c7

    elif(list_of_operations[0]=='AND' or list_of_operations[0]=='and'):
      res,c8=xandy(res,list_wordnext)
      total_comparisons=total_comparisons+c8

  #Print the nummber of documents matched and the list of documents which contain the query

  print("Number of documents matched: ",len(res))

  #Print the number of comparisons required
  print("No. of comparisons required: ", total_comparisons)
  if(len(res)!=0):
      for i in range(len(res)):
        print("doc name ::  ",doc_docID_mapping[res[i]])
  else:
      print("No matching Document!!!")

  
#Out comes a punk at dusk


Please enter the number of queries ::  1
Please enter the query ::  fuck you
['fuck']
Only Single word in the query, hence no further operation needed
Number of Documents retrieved is ::   45
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/dirtword.txt
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/insanity.hum
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/lawsuniv.hum
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/killself.hum
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/looser.hum
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/losers86.hum
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/cars.txt
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/reagan.hum
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/donut.txt
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/fuckyou2.txt
doc name ::   /content/drive/MyDrive/Humor,Hist,Media,Food/terbear.txt
doc name ::   /content/drive

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


**Question 2(A) and Question 2(B)**

In [ ]:
#This method takes the data files as input and preprocesses them and creates a Positional index - Positional Index is a dictionary of dictionary-
#The key is the word- which has a key of DOCID and then a list of positions in respective documents as the values
import os
def preprocess_and_positional_index():
  word_dict_temp=dict()

  directory = '/content/drive/MyDrive/Humor,Hist,Media,Food'

  count=0
  docID=1
  #Read the data files
  for filename in os.scandir(directory):
      flag=0
      if filename.is_file():
          #print(filename.path)
          ccfile = open(filename.path, "r",encoding="utf8", errors="surrogateescape")
          counter_word_position=1
        
          #preprocessing on the data
          for aline in ccfile:
              #Convert to lowercase
              aline = aline.lower()
              #Remove numerics
              aline = re.sub(r'\d+','',aline)
              #Remove whitespaces
              aline = re.sub(r'[^\w\s]','', aline)
              #Remove special characters
              aline= re.sub('[@_!#$%^&*()<>?/\|}{~:]','',aline)
              aline = aline.strip()
              #Remove stopwords
              aline = remove_stopwords(aline)
              #Tokenize the words
              words = word_tokenize(aline)



              for w in words:
                  #If the word doesnt already exist in dictionary then add it
                  if ps.stem(w) not in word_dict_temp:
                    dict1=dict()
                    dict1[docID]=[counter_word_position]
                    word_dict_temp[ps.stem(w)]=dict1
                    counter_word_position=counter_word_position+1
                  #If the documentId of the word doesnt exist in dictionary then add the docId and the position
                  else:
                    dict_temp=word_dict_temp.get(ps.stem(w))
                    if docID not in dict_temp:
                      word_dict_temp[ps.stem(w)][docID]=[counter_word_position]
                    #If docid already exists just append the position of the word in the list
                    else:
                      word_dict_temp[ps.stem(w)][docID].append(counter_word_position)
                    
                    counter_word_position=counter_word_position+1
                    

                  #print(ps.stem(w))
          
          ccfile.close()
          
      docID=docID+1
  #Return the index
  return word_dict_temp                


In [ ]:
word_dict_temp=dict()
word_dict_temp=preprocess_and_positional_index()

In [ ]:
#This method finds intersection on two lists
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

#Preprocessing on word
def preprocess(word):
      #Convert to lowercase
      word = word.lower()
      #Remove numerics
      word = re.sub(r'\d+','',word)
      #Remove whitespaces
      word = re.sub(r'[^\w\s]','', word)
      #Remove special characters
      word= re.sub('[@_!#$%^&*()<>?/\|}{~:]','',word)
      word = word.strip()
      #Remove stop words
      word = remove_stopwords(word)
      #Return the Root word- stemming
      return ps.stem(word)

In [ ]:
#This block takes a Phrase query from user, performs preprocessing on it and return the doc in which the phrase is present
#It return the number of docs and the names of documents in which the phrase is present

#Take the number of phrase queries as input
total_number_phrase=int(input("Please enter the number of queries ::  "))
#word_dict_temp=dict()
#word_dict_temp=preprocess_and_positional_index()

#Take the phrase query
for i in range(total_number_phrase):
  query=input("Please enter the query ::  ")
  word_dict_temp=dict()
  word_dict_temp=preprocess_and_positional_index()
  query=query.split()

  #Preform preprocessing on the query
  list_words=[]
  for word in query:
        #Convert to lowercase
        word = word.lower()
        #Remove numerics
        word = re.sub(r'\d+','',word)
        #Remove whitespaces
        word = re.sub(r'[^\w\s]','', word)
        #Remove special characters
        word= re.sub('[@_!#$%^&*()<>?/\|}{~:]','',word)
        word = word.strip()
        #Remove stopwords
        word = remove_stopwords(word)

        if word!='':
          list_words.append(word)

  prerocessed_list_of_words=[]
  for i in range(len(list_words)):
    prerocessed_list_of_words.append(preprocess(list_words[i]))

  lists=[]

  for i in range(len(prerocessed_list_of_words)):
    temp_dict=word_dict_temp.get(prerocessed_list_of_words[i])
    lists.append(temp_dict)

  result=lists[0].keys();

  for i in range(1,len(prerocessed_list_of_words)):

    #handle scenario where the word doesnot exist in index
    if lists[i] is None:
      print("Cannot find the word ", prerocessed_list_of_words[i],"in the dictionary, hence cannot process further.")
      sys.exit("Cannot find the word in the dictionary")
    
    result=result & lists[i].keys()

  result=list(result)
  number_of_matched_docs=0
  flag=0
  #Iterate on the list of words, Get the common doc ids to which they belong- Get the positions of the words in the common doc
  for i in range(len(result)):
    positons_in_doc=[]

    #check if the next word in the phrase query is the next word in the document and so on..
    for j in range(len(prerocessed_list_of_words)):
      tempdict=word_dict_temp.get(prerocessed_list_of_words[j])
      tempwordpos=tempdict.get(result[i])
      positons_in_doc.append(tempwordpos)


      for k in range(len(tempwordpos)):
        tempwordpos[k]=tempwordpos[k]-j;

    

    #Create final position list- 
    final_positions=positons_in_doc[0]

    for l in range(1,len(positons_in_doc)):
      #print(final_positions)
      #print(positons_in_doc[l])
      final_positions=intersection(final_positions,positons_in_doc[l])
      #print(final_positions)
      #print()


    #print(final_positions)

    #Print the doc names in which the phrase is present
    if(len(final_positions)>=1):
      flag=1
      #print("doc ID is::   ",result[i])
      print("docName is::  ",doc_docID_mapping[result[i]])
      number_of_matched_docs=number_of_matched_docs+1
      


  #Print the number of documents in which the phrase is present
  print("Number of documents recieved :: ",number_of_matched_docs)
  if flag==0:
    print("no matching doc found")

  



Please enter the number of queries ::  1
Please enter the query ::  Flies can be found around window sills
docName is::   /content/drive/MyDrive/Humor,Hist,Media,Food/bugbreak.hum
docName is::   /content/drive/MyDrive/Humor,Hist,Media,Food/how.bugs.breakd
docName is::   /content/drive/MyDrive/Humor,Hist,Media,Food/bugs.txt
Number of documents recieved ::  3
